# Backpropagation through Back substitution with a Backslash
Notebook per il seminario d'esame per il corso di Metodi di Approssimazione 

## Algebra degli operatori

Definiamo il tipo di dato custom che rappresenta la funzione lineare

In [2]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [3]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator

adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪()), X::Matrix) = -X  # 𝒪() - X should be -X
\(ℐ::typeof(ℐ()), A::Matrix) = A    # left division with ℐ() does nothing
*(A::Operator, X::Matrix) = A.op(X)
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

E alcune regole create da me, ma che non attivo ancora

## Esempi

Vediamo qualche esempio.

In [6]:
# Basic Test
B = [ 1 2; 3 4]
M = [10 1;1 10]
C = [ 2 5; 4 6]

LM = ℒ(M)

ℒ(2, 2)

### `Operator * Matrix`

A parte la negazione `-`, la prima regola che abbiamo scritto è
```
*(A::Operator, X::Matrix) = A.op(X)
```
Ossia, il prodotto tra un operatore e una matrice consiste nell'applicare l'operatore alla matrice. 

Iniziamo guardando la moltiplicazione a sinistra

In [7]:
typeof(LM), typeof(I(2))

(Operator, Diagonal{Bool, Vector{Bool}})

In [8]:
LM * [1 0; 0 1]

2×2 Matrix{Int64}:
 10   1
  1  10

Osserviamo che abbiamo definito il prodotto *a destra* per una matrice. Se moltiplichiamo a sinistra per una matrice, le cose non tornano 

In [9]:
try
    [1 0; 0 1] * LM
catch e
    print("Non si può fare questa moltiplicazione. Ecco l'errore che dà:\n\n")
    showerror(stdout, e)
end

Non si può fare questa moltiplicazione. Ecco l'errore che dà:

MethodError: no method matching *(::Matrix{Int64}, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::AbstractMatrix, ::LinearAlgebra.AbstractQ)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/abstractq.jl:201
  *(::AbstractMatrix, ::LinearAlgebra.AbstractRotation)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/givens.jl:20
  ...


... e ci sono altri piccoli problemini per il fatto che abbiamo usato `Matrix` 

In [10]:
try
    LM * I(2)
catch e
    print("No, non si può fare:\n\n")
    showerror(stdout, e)
end

No, non si può fare:

MethodError: no method matching *(::Operator, ::Diagonal{Bool, Vector{Bool}})
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::Operator, ::Matrix)
   @ Main ~/Scrivania/MA-project/Codice/src/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:11
  *(::Transpose{T, <:AbstractVector} where T, ::Diagonal, ::AbstractVector)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/diagonal.jl:1034
  ...


Non si può fare perché `I` non è una `Matrix` bensì una `AbstractMatrix`:

In [11]:
typeof(I(2)) <: Matrix, typeof(I(2)) <: AbstractMatrix

(false, true)

Ma torniamo alle cose che funzionano

In [12]:
LM * B

2×2 Matrix{Int64}:
 13  24
 31  42

E in effetti è la stessa cosa che fare...

In [13]:
LM.op(B)

2×2 Matrix{Int64}:
 13  24
 31  42

Vediamo ora la moltiplicazione a destra

In [14]:
ℛ(M) * B # i.e. ℛ(M).op(B) 

2×2 Matrix{Int64}:
 12  21
 34  43

In [15]:
B * M # right multiply by M

2×2 Matrix{Int64}:
 12  21
 34  43

La notazione appare controintuitiva, perché lo stesso simbolo `*` viene usato per denotare la moltiplicazione di matrici, e in effetti alla fin fine di un prodotto tra matrici si tratta. Eppure ha senso in realtà: `ℛ(M) * B ` denota l'applicazione dell'operatore `ℛ(M)` a `B`, ovvero `ℛ(M).op(B)`.

Per completezza, vediamo il prodotto di Hadamard

In [16]:
[ℋ(M) * B M .* B]

2×4 Matrix{Int64}:
 10   2  10   2
  3  40   3  40

Ricordiamo che in Julia `.*` vuol dire moltiplicazione *pointwise*

### Trasposizione di operatori

Tra le varie cose, abbiamo scritto
```
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
```
Ovvero abbiamo definito la trasposizione tra operatori. Infatti ricordiamo che in Julia `A'` essenzialmente viene tradotto in `adjoint(A)`.

Ricordiamo inoltre che `adjoint` 
- Fa l'aggiunta "**lazy**". Non "materializza" alcun nuovo oggetto, bensì restituisce una "view" differente sull'oggetto a cui viene applicato
    - Se `B = A'`, modificando `B` si modifica `A`
- E' **involutorio**. Questo "wrapper" viene rimosso quando il metodo viene chiamato due volte, restituendo il "genitore", i.e. l'elemento che è stato trasposto
    - `A'` registra chi è il genitore (`A`) e il tipo di elementi che restituisce dopo la trasposizione (i.e. aggiunto di `eltype(A)`)
- E' **ricorsivo**. `A'` traspone `A` e applica il metodo `adjoint` agli elementi di `A`
    - Concettualmente, `adjoint(A)[i,j] = adjoint(A[j,i])`.



Verifichiamo che la trasposizione di operatori faccia il suo dovere, ossia che 
$$\langle B, \mathcal{L}\,C\rangle=\langle\mathcal{L}'\,B, C\rangle$$
dove $\mathcal{L}'$ denota l'operatore aggiunto $\mathcal{L}^{\top}$

In [17]:
print("<B,ℒC> = $(tr( B'*(ℒ(M)*C) )),\t <ℒ'B,C> = $(tr( (ℒ(M)'*B)' * C ))")   # <ℒ'B,C>

<B,ℒC> = 522,	 <ℒ'B,C> = 522

In termini un po' più "espliciti"

In [18]:
tr(B' * ℒ(M).op(C)), tr( ℒ(M).adj(B)' * C)

(522, 522)

### `Operator ∘ Operator`

Un'altra importante regola che abbiamo scritto è
```
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
```
Che definisce la composizione tra operatori

La composizione fa da prodotto nell'algebra degli endomorfismi di uno spazio vettoriale. 

Quindi potrebbe essere ragionevole denotare la composizione con `*`.

***OSS***: purtroppo non lavoriamo con *endomorfismi* di $\mathbb{R}^{m\times n}$, bensì con omomorfismi tra spazi di matrici. Quindi per essere "sound" dovremmo mettere un controllo sull'ammissibilità dell'operazione. Invece così com'è ora è possibile definire composizioni di omomorfismi che non si possono comporre, e scoprire l'errore solo a runtime.

Comunque, i pezzi di codice sono puramente strumentali a far funzionare il metodo che viene presentato nell'articolo. Questo è soltanto un primo esempio di questo fenomeno. Nella fattispecie, la composizione viene usata solo per costruire gli elementi delle matrici `M` e `ImL`. Quindi, sì, potremmo anche sovraccaricare il prodotto tra omomorfismi...

In [19]:
try
    comp = ℒ(M) * ℒ(M)
catch e
    print("Errore!\n\n")
    showerror(stdout, e)
    println()
end

Errore!

MethodError: no method matching *(::Operator, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::Operator, ::Matrix)
   @ Main ~/Scrivania/MA-project/Codice/src/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:11
  *(::BitMatrix, ::UniformScaling)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/uniformscaling.jl:264
  ...



In [20]:
comp1 = ℒ(M) ∘ ℒ(M)

ℒ(2, 2)∘ℒ(2, 2)

In [21]:
comp1 * [1 0; 0 1]

2×2 Matrix{Int64}:
 101   20
  20  101

fa ciò che ci si aspetta:

In [22]:
(M * M) * [1 0; 0 1]

2×2 Matrix{Int64}:
 101   20
  20  101

Vediamo un altro esempio

In [23]:
comp2 = ℒ(M) ∘ ℛ(C)

ℒ(2, 2)∘ℛ(2, 2)

In [24]:
comp2 * B       # (ℒ(M) ∘ ℛ(C))[B]

2×2 Matrix{Int64}:
 122  209
 230  407

Sarebbe

In [25]:
ℒ(M).op(ℛ(C).op(B))

2×2 Matrix{Int64}:
 122  209
 230  407

ovvero

In [26]:
M * B * C

2×2 Matrix{Int64}:
 122  209
 230  407

#### Non controllo di ammissibilità

**Attenzione!** Come già accennato, potremmo star puntando la pistola contro il nostro piede, e il sistema ci avviserebbe che premendo il grilletto il proiettile parte contro il nostro corpo solo dopo che il mignolino è saltato per aria!

In termini meno poetici, potremmo star componendo omomorfismi che non si possono comporre, e il sistema ce lo lascerebbe fare

In [27]:
# Julia ci lascia inserire il proiettile...
compWrong = ℒ([1 2 3]) ∘ ℒ([2 -1; 1 2])

ℒ(1, 3)∘ℒ(2, 2)

In [28]:
# ... e ci avverte solo dopo lo sparo!
try 
    compWrong * rand(2,3)
catch e
    print("Sparandoti contro il piede, ti farai male al piede!\n\n")
    print("Come dici? Potevo dirtelo prima? Avrei dovuto non lasciarti puntare il fucile?\n")
    print("Beh, e che ne sapevo che avevi intenzione di sparare?!?")
end

Sparandoti contro il piede, ti farai male al piede!

Come dici? Potevo dirtelo prima? Avrei dovuto non lasciarti puntare il fucile?
Beh, e che ne sapevo che avevi intenzione di sparare?!?

#### Sovraccaricare `Operator * Operator`

Aggiungendo la regola

In [29]:
*(A::Operator, B::Operator) = A ∘ B

* (generic function with 191 methods)

Riusciamo a scrivere la composizione di operatori come prodotto

In [30]:
A1 = [1 2; -2 1]
A2 = [0 3 1; 2 0 -1]

comp3 = ℒ(A1) * ℒ(A2)

ℒ(2, 2)∘ℒ(2, 3)

In [ ]:
display(comp3 * [1 0 0; 0 1 0; 0 0 1])  # (ℒ(A1) ∘ ℒ(A2))[I(3)]
display(A1 * A2)

2×3 Matrix{Int64}:
 4   3  -1
 2  -6  -3

2×3 Matrix{Int64}:
 4   3  -1
 2  -6  -3

Funziona